# Analysis

In [2]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [192]:
from src.data_loading.psl import get_psl_indices

In [193]:
ds = get_psl_indices()

In [195]:
import matplotlib.pyplot as plt
import seaborn as sns
df = ds.to_dataframe()
# alt_pairplot(df)

In [137]:
from src.data_loading.psl import get_ersstv5
ersstv5 = get_ersstv5()

In [204]:
ersstv5_slim = ersstv5.sel(time=slice('1948', '2021'))

In [160]:
from src.xr_utils import can_coords, fix_calendar, sel
m, clim =nino_calculate(fix_calendar(can_coords(ersstv5_slim)))

warning, not changing:  variable


In [205]:
ersstv5_can = fix_calendar(can_coords(ersstv5_slim)).isel(variable=0)

In [223]:
for x, y in [("nino34", "nino3.4"), ("nino3", "nino3"), ("nino4", "nino4"), ("nino1+2", "nino1+2")]:
    m, clim =nino_calculate(ersstv5_can, reg=y)
    my_nino = m.values
    psl_nino = ds[x].sel(time=slice("1948", "2021-10")).values
    print(ma.corrcoef(ma.masked_invalid(my_nino), ma.masked_invalid(psl_nino)))

[[1.0 0.9930262268868004]
 [0.9930262268868004 1.0]]
[[1.0 0.990935864214245]
 [0.990935864214245 1.0]]
[[1.0 0.9925296993942325]
 [0.9925296993942325 1.0]]
[[1.0 0.9820494375705351]
 [0.9820494375705351 1.0]]
